# Customer Churn Analysis & Prediction

## Project Overview

This notebook analyzes customer churn patterns in telecom data, identifies key drivers of customer attrition,
and builds predictive models to help reduce churn rates and increase customer retention.

**Key Business Question**: What factors influence customer churn and how can we predict at-risk customers?


## 1.0 Environment Setup


### 1.1 Library Imports

Standard libraries


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import time
from pathlib import Path
import joblib

Interactive visualization libraries


In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import missingno as msno

Machine learning libraries


In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    StratifiedKFold,
    GridSearchCV,
)
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_curve,
    auc,
    precision_recall_curve,
    average_precision_score,
    f1_score,
    roc_auc_score,
)

Import models


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

Clustering and dimensionality reduction


In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

### 1.2 Configure Environment


In [ ]:
plt.style.use("seaborn-v0_8-whitegrid")

In [ ]:
custom_palette = [
    "#4e79a7",
    "#f28e2b",
    "#e15759",
    "#76b7b2",
    "#59a14f",
    "#edc948",
    "#b07aa1",
    "#ff9da7",
    "#9c755f",
    "#bab0ac",
]

sns.set_palette(custom_palette)

In [ ]:
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 12
warnings.filterwarnings("ignore")

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

### 1.3 Project Configuration


In [ ]:
CONFIG = {
    "random_seed": 42,
    "test_size": 0.25,
    "cv_folds": 5,
    "data_path": "../data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv",
    "figures_path": "../reports/figures/",
    "models_path": "../models/",
    "results_path": "../reports/results/",
    "target_column": "Churn",
    "positive_class": "Yes",
    "id_column": "customerID",
}

In [ ]:
for path in [CONFIG["figures_path"], CONFIG["models_path"], CONFIG["results_path"]]:
    Path(path).mkdir(parents=True, exist_ok=True)

Set random seed for reproducibility


In [ ]:
np.random.seed(CONFIG["random_seed"])

### 1.4 Utility Functions


In [ ]:
def save_fig(fig: plt.Figure, filename: str, dpi: int = 300) -> None:
    """
    Save a matplotlib figure with a standardized format.

    Args:
      fig (plt.Figure): The matplotlib figure to save.
      filename (str): The name of the file to save the figure as.
      dpi (int, optional): The resolution of the saved figure in dots per inch. Default is 300.

    Returns:
      None: This function does not return any value. It saves the figure to the specified path.
    """
    full_path = Path(CONFIG["figures_path"]) / filename
    fig.savefig(full_path, bbox_inches="tight", dpi=dpi)

    print(f"Figure saved: {full_path}")

In [ ]:
def format_runtime(seconds):
    """
    Format runtime in human-readable format
    """

    if seconds < 60:
        return f"{seconds:.2f} seconds"

    elif seconds < 3600:
        minutes = seconds / 60
        return f"{minutes:.2f} minutes"

    else:
        hours = seconds / 3600
        return f"{hours:.2f} hours"

Track execution time


In [ ]:
project_start_time = time.time()

In [ ]:
print("Environment setup complete.")
print(
    f"Project initialized with pandas {pd.__version__}, scikit-learn and modern visualization libraries."
)

## 2.0 Data Loading and Overview


In [ ]:
load_start = time.time()

df = pd.read_csv(CONFIG["data_path"])
load_time = time.time() - load_start

In [ ]:
print(f"Dataset loaded in {format_runtime(load_time)}")
print(f"Dataset dimensions: {df.shape[0]:,} rows, {df.shape[1]:,} columns")

Display first few rows


In [ ]:
display(df.head())

Data types and basic statistics


In [ ]:
data_types = pd.DataFrame(
    {
        "Type": df.dtypes,
        "Non-Null Count": df.count(),
        "Null Count": df.isnull().sum(),
        "Null %": (df.isnull().sum() / len(df) * 100).round(2),
        "Unique Values": df.nunique(),
    }
)

display(data_types)

Check for potential numeric columns stored as strings


In [ ]:
numeric_as_object = []
for col in df.select_dtypes(include=["object"]).columns:
    if pd.to_numeric(df[col], errors="coerce").notna().mean() > 0.9:
        numeric_as_object.append(col)

if numeric_as_object:
    print(f"\nColumns that might need type conversion: {numeric_as_object}")

### 2.1 Visualize Missing Values


In [ ]:
plt.figure(figsize=(12, 6))
msno.matrix(df)

plt.title("Missing Values Matrix", fontsize=16)
plt.tight_layout()
plt.show()

### 2.2 Class Imbalance Checks


In [ ]:
target_col = CONFIG["target_column"]
target_counts = df[target_col].value_counts()
target_pct = df[target_col].value_counts(normalize=True) * 100

In [ ]:
fig = px.bar(
    x=target_counts.index,
    y=target_counts.values,
    color=target_counts.index,
    text=target_counts.values,
    labels={"x": CONFIG["target_column"], "y": "Count"},
    title=f'Distribution of Target Variable: {CONFIG["target_column"]}',
    color_discrete_map={CONFIG["positive_class"]: "#e74c3c", "No": "#3498db"},
    template="plotly_white",
)

fig.update_layout(
    showlegend=False,
    title_font_size=20,
    title_x=0.5,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14,
)

fig.update_traces(
    texttemplate="%{text:,}", textposition="outside", textfont=dict(size=14)
)

fig.update_layout(
    annotations=[
        dict(
            x=i,
            y=count + (max(target_counts.values) * 0.05),
            text=f"{pct:.1f}%",
            showarrow=False,
            font=dict(size=14),
        )
        for i, (count, pct) in enumerate(zip(target_counts.values, target_pct.values))
    ]
)

fig.show()

In [ ]:
print(f"Churn rate: {target_pct[CONFIG['positive_class']]:.2f}%")
print(
    f"This represents {target_counts[CONFIG['positive_class']]:,} out of {len(df):,} customers."
)

### 2.3 Identify Feature Types


In [ ]:
id_cols = [CONFIG["id_column"]]
target_cols = [CONFIG["target_column"]]
categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
numerical_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()

In [ ]:
for col in id_cols + target_cols:
    if col in categorical_cols:
        categorical_cols.remove(col)

In [ ]:
print("\nFeature Classification:")
print(f"ID Columns ({len(id_cols)}): {', '.join(id_cols)}")
print(f"Target Column: {', '.join(target_cols)}")
print(f"Categorical Features ({len(categorical_cols)}): {', '.join(categorical_cols)}")
print(f"Numerical Features ({len(numerical_cols)}): {', '.join(numerical_cols)}")

## 3.0 Data Cleaning and Preprocessing


In [ ]:
df_clean = df.copy()

Convert SeniorCitizen from numeric to categorical


In [ ]:
df_clean["SeniorCitizen"] = df_clean["SeniorCitizen"].map({0: "No", 1: "Yes"})
categorical_cols.append("SeniorCitizen")
numerical_cols.remove("SeniorCitizen")

Handle TotalCharges column if it's stored as object


In [ ]:
if df_clean["TotalCharges"].dtype == "object":
    df_clean["TotalCharges"] = pd.to_numeric(df_clean["TotalCharges"], errors="coerce")
    print(
        f"✓ Converted 'TotalCharges' to numeric. Found {df_clean['TotalCharges'].isnull().sum()} missing values."
    )

    if df_clean["TotalCharges"].isnull().sum() > 0:
        print("\nRows with missing TotalCharges:")
        display(df_clean[df_clean["TotalCharges"].isnull()])

        mask = df_clean["TotalCharges"].isnull()
        df_clean.loc[mask, "TotalCharges"] = (
            df_clean.loc[mask, "MonthlyCharges"] * df_clean.loc[mask, "tenure"]
        )

        print(
            f"✓ Filled {mask.sum()} missing values in 'TotalCharges' based on MonthlyCharges × tenure"
        )

Validate data integrity after cleaning


In [ ]:
remaining_missing = df_clean.isnull().sum().sum()
print(f"\nRemaining missing values after cleaning: {remaining_missing}")

if remaining_missing == 0:
    print("✅ All missing values have been successfully handled.")

else:
    print("⚠️ There are still missing values that need to be addressed.")

Data summary after cleaning


In [ ]:
display(df_clean.describe())

In [ ]:
df_clean.to_csv(
    Path(CONFIG["data_path"]).parent / "cleaned_churn_data.csv", index=False
)

## 4.0 Exploratory Data Analysis

### 4.1 Categorical Feature Analysis


In [ ]:
def analyze_categorical_features(
    df: pd.DataFrame, feature: pd.Series, target: pd.Series, title: str = None
):
    """
    Create a comprehensive analysis of a categorical feature.

    Args:
      df (pandas.DataFrame): DataFrame containing the data
      feature (pandas.Series): Column name of the categorical feature
      target (pandas.Series): Column name of the target variable
      title (str): Title for the plot, defaults to feature name
    """

    if title is None:
        title = f"Analysis of {feature}"

    value_counts = df[feature].value_counts().reset_index()
    value_counts.columns = [feature, "Count"]
    value_counts["Percentage"] = (value_counts["Count"] / len(df) * 100).round(1)

    # Calculate churn rate by category
    churn_by_category = (
        df.groupby(feature)[target]
        .apply(lambda x: (x == CONFIG["positive_class"]).mean() * 100)
        .reset_index()
    )
    churn_by_category.columns = [feature, "Churn Rate (%)"]

    # Sort by churn rate
    churn_by_category = churn_by_category.sort_values("Churn Rate (%)", ascending=False)

    # Subplot with two charts: distribution bar chart and churn rate bar chart
    fig = make_subplots(
        rows=1,
        cols=2,
        subplot_titles=["Distribution by Category", "Churn Rate by Category"],
        specs=[[{"type": "bar"}, {"type": "bar"}]],
        column_widths=[0.5, 0.5],
    )

    sorted_counts = value_counts.sort_values("Count", ascending=False)
    fig.add_trace(
        go.Bar(
            x=sorted_counts[feature],
            y=sorted_counts["Count"],
            text=[
                f"{count:,} ({pct}%)"
                for count, pct in zip(
                    sorted_counts["Count"], sorted_counts["Percentage"]
                )
            ],
            textposition="auto",
            marker_color="#3498db",
            name="Count",
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Bar(
            x=churn_by_category[feature],
            y=churn_by_category["Churn Rate (%)"],
            text=[f"{rate:.1f}%" for rate in churn_by_category["Churn Rate (%)"]],
            textposition="auto",
            marker_color=churn_by_category["Churn Rate (%)"],
            marker=dict(
                colorscale="RdYlGn_r",
                colorbar=dict(title="Churn %"),
            ),
            name="Churn Rate (%)",
        ),
        row=1,
        col=2,
    )

    fig.update_layout(
        title=title,
        height=500,
        width=1000,
        template="plotly_white",
        showlegend=False,
        title_font_size=18,
        title_x=0.5,
    )

    fig.update_xaxes(title_text="", tickangle=45, row=1, col=1)
    fig.update_xaxes(title_text="", tickangle=45, row=1, col=2)
    fig.update_yaxes(title_text="Count", row=1, col=1)
    fig.update_yaxes(title_text="Churn Rate (%)", row=1, col=2)

    fig.show()

    # Perform chi-square test
    contingency_table = pd.crosstab(df[feature], df[target])
    chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

    # Cramer's V for effect size
    n = contingency_table.sum().sum()
    cramer_v = np.sqrt(chi2 / (n * (min(contingency_table.shape) - 1)))

    # Interpret effect size
    if cramer_v < 0.1:
        effect = "negligible"

    elif cramer_v < 0.2:
        effect = "weak"

    elif cramer_v < 0.3:
        effect = "moderate"

    else:
        effect = "strong"

    print(f"Statistical Analysis for {feature}:")
    print(f"- Chi-Square Test: χ² = {chi2:.2f}, p-value = {p:.4f}")
    print(f"- Effect Size (Cramer's V): {cramer_v:.4f} - {effect} effect")

    if p < 0.05:
        print(f"- Conclusion: Statistically significant association with churn")

    else:
        print(f"- Conclusion: No statistically significant association with churn")

    return chi2, p, cramer_v

#### 4.1.1 Analyze categorical features


In [ ]:
cat_results = []
for col in categorical_cols:
    print(f"\nAnalyzing {col}:")
    chi2, p, cramer_v = analyze_categorical_features(
        df_clean, col, CONFIG["target_column"]
    )

    cat_results.append(
        {"Feature": col, "Chi2": chi2, "P-value": p, "Cramer_V": cramer_v}
    )

#### 4.1.2 Summary of Categorical Features


In [ ]:
cat_summary = pd.DataFrame(cat_results).sort_values("Cramer_V", ascending=False)
cat_summary["Significant"] = cat_summary["P-value"] < 0.05
cat_summary["Effect Size"] = cat_summary["Cramer_V"].apply(
    lambda v: (
        "Strong"
        if v >= 0.3
        else "Moderate" if v >= 0.2 else "Weak" if v >= 0.1 else "Negligible"
    )
)

In [ ]:
display(cat_summary)

#### 4.1.3 Visualize Categorical Feature Importance


In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(
    x="Cramer_V",
    y="Feature",
    hue="Significant",
    data=cat_summary.sort_values("Cramer_V"),
    palette=["#e74c3c", "#3498db"],
)

plt.title("Categorical Feature Importance by Effect Size", fontsize=16)
plt.xlabel("Effect Size (Cramer's V)", fontsize=14)
plt.ylabel("Feature", fontsize=14)
plt.legend(title="Statistically Significant")
plt.tight_layout()
plt.show()

### 4.2 Numerical Feature Analysis


In [ ]:
def analyze_numerical_features(
    df: pd.DataFrame, feature: pd.Series, target: pd.Series, title: str = None
):
    """
    Create a comprehensive analysis of a numerical feature

    Args:
      df (pandas.DataFrame): DataFrame containing the data
      feature (pandas.Series): Column name of the numerical feature
      target (pandas.Series): Column name of the target variable
      title (str): Title for the plot, defaults to feature name
    """

    try:
        if title is None:
            title = f"Analysis of {feature}"

        if df[feature].isnull().sum() > 0:
            print(
                f"Warning: {feature} contains {df[feature].isnull().sum()} missing values"
            )

            df = df.copy()
            df[feature] = df[feature].fillna(df[feature].median())

        churned = df[df[target] == CONFIG["positive_class"]][feature]
        retained = df[df[target] != CONFIG["positive_class"]][feature]

        if len(churned) < 2 or len(retained) < 2:
            print(
                f"Error: Not enough data for {feature} analysis. Churned: {len(churned)}, Retained: {len(retained)}"
            )

            return None

        # Subplot
        fig = make_subplots(
            rows=2,
            cols=2,
            subplot_titles=[
                "Distribution by Churn Status",
                "Box Plot by Churn Status",
                "Mean Comparison with 95% CI",
                "Cumulative Distribution Function",
            ],
            specs=[
                [{"type": "histogram"}, {"type": "box"}],
                [{"type": "bar"}, {"type": "scatter"}],
            ],
            vertical_spacing=0.1,
        )

        # Add histogram for distribution comparison
        fig.add_trace(
            go.Histogram(
                x=churned, name="Churned", opacity=0.7, marker_color="#e74c3c"
            ),
            row=1,
            col=1,
        )

        fig.add_trace(
            go.Histogram(
                x=retained, name="Retained", opacity=0.7, marker_color="#3498db"
            ),
            row=1,
            col=1,
        )

        # Add box plots
        fig.add_trace(
            go.Box(y=churned, name="Churned", marker_color="#e74c3c", boxmean=True),
            row=1,
            col=2,
        )

        fig.add_trace(
            go.Box(y=retained, name="Retained", marker_color="#3498db", boxmean=True),
            row=1,
            col=2,
        )

        def mean_confidence_interval(data, confidence: float = 0.95):
            a = 1.0 * np.array(data)
            n = len(a)

            m, se = np.mean(a), stats.sem(a)
            h = se * stats.t.ppf((1 + confidence) / 2.0, n - 1)

            return m, m - h, m + h

        churn_mean, churn_lower, churn_upper = mean_confidence_interval(churned)
        retain_mean, retain_lower, retain_upper = mean_confidence_interval(retained)

        # Create bar chart with error bars
        fig.add_trace(
            go.Bar(
                x=["Churned", "Retained"],
                y=[churn_mean, retain_mean],
                error_y=dict(
                    type="data",
                    array=[churn_upper - churn_mean, retain_upper - retain_mean],
                    arrayminus=[churn_mean - churn_lower, retain_mean - retain_lower],
                    visible=True,
                ),
                marker_color=["#e74c3c", "#3498db"],
                text=[f"{churn_mean:.2f}", f"{retain_mean:.2f}"],
                textposition="auto",
            ),
            row=2,
            col=1,
        )

        def ecdf(data):
            """
            Compute ECDF for a one-dimensional array of measurements
            """

            x = np.sort(data)
            n = len(x)
            y = np.arange(1, n + 1) / n

            return x, y

        x_churned, y_churned = ecdf(churned)
        x_retained, y_retained = ecdf(retained)

        fig.add_trace(
            go.Scatter(
                x=x_churned,
                y=y_churned,
                mode="lines",
                name="Churned",
                line=dict(color="#e74c3c", width=2),
            ),
            row=2,
            col=2,
        )

        fig.add_trace(
            go.Scatter(
                x=x_retained,
                y=y_retained,
                mode="lines",
                name="Retained",
                line=dict(color="#3498db", width=2),
            ),
            row=2,
            col=2,
        )

        fig.update_layout(
            title=title,
            height=700,
            width=1000,
            template="plotly_white",
            showlegend=True,
            barmode="overlay",
            title_font_size=18,
            title_x=0.5,
            legend=dict(
                orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1
            ),
        )

        fig.update_xaxes(title_text=feature, row=1, col=1)
        fig.update_yaxes(title_text="Count", row=1, col=1)
        fig.update_yaxes(title_text=feature, row=1, col=2)
        fig.update_xaxes(title_text="Churn Status", row=2, col=1)
        fig.update_yaxes(title_text=f"Mean {feature}", row=2, col=1)
        fig.update_xaxes(title_text=feature, row=2, col=2)
        fig.update_yaxes(title_text="Cumulative Probability", row=2, col=2)

        fig.show()

        # Statistical Tests
        try:
            u_stat, p_value = stats.mannwhitneyu(churned, retained)
            mean_diff = churned.mean() - retained.mean()
            pooled_std = np.sqrt((churned.std() ** 2 + retained.std() ** 2) / 2)
            cohens_d = abs(mean_diff / pooled_std)

            if cohens_d < 0.2:
                effect = "negligible"

            elif cohens_d < 0.5:
                effect = "small"

            elif cohens_d < 0.8:
                effect = "medium"

            else:
                effect = "large"

            print(f"Statistical Analysis for {feature}:")
            print(f"- Mann-Whitney U Test: U = {u_stat:.2f}, p-value = {p_value:.4f}")
            print(f"- Effect Size (Cohen's d): {cohens_d:.4f} - {effect} effect")

            if p_value < 0.05:
                print(
                    f"- Conclusion: Statistically significant difference between churned and retained customers"
                )

            else:
                print(
                    f"- Conclusion: No statistically significant difference between churned and retained customers"
                )

            # Descriptive stats
            print(f"\nDescriptive Statistics:")
            print(
                f"- Churned Customers: mean = {churned.mean():.2f}, median = {churned.median():.2f}, std = {churned.std():.2f}"
            )

            print(
                f"- Retained Customers: mean = {retained.mean():.2f}, median = {retained.median():.2f}, std = {retained.std():.2f}"
            )

            print(f"- Difference in means: {(churned.mean() - retained.mean()):.2f}")

            return u_stat, p_value, cohens_d

        except Exception as e:
            print(f"Error performing statistical tests for {feature}: {str(e)}")

    except Exception as e:
        print(f"Error analyzing {feature}: {str(e)}")
        return None

#### 4.2.1 Analyze numerical features


In [ ]:
num_results = []
for col in numerical_cols:
    print(f"\nAnalyzing {col}:")
    result = analyze_numerical_features(df_clean, col, CONFIG["target_column"])

    if result is not None:
        u_stat, p_value, cohens_d = result

    else:
        print(f"Skipping analysis for {col} due to missing or invalid data.")
        continue

    num_results.append(
        {"Feature": col, "U_stat": u_stat, "P-value": p_value, "Cohens_d": cohens_d}
    )

#### 4.2.2 Create a Summary of Numerical Feature Importance


In [ ]:
num_summary = pd.DataFrame(num_results).sort_values("Cohens_d", ascending=False)
num_summary["Significant"] = num_summary["P-value"] < 0.05
num_summary["Effect Size"] = num_summary["Cohens_d"].apply(
    lambda d: (
        "Large"
        if d >= 0.8
        else "Medium" if d >= 0.5 else "Small" if d >= 0.2 else "Negligible"
    )
)

In [ ]:
display(num_summary)

#### 4.2.3 Numerical Feature Importance


In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(
    x="Cohens_d",
    y="Feature",
    hue="Significant",
    data=num_summary.sort_values("Cohens_d"),
    palette=["#e74c3c", "#3498db"],
)

plt.title("Numerical Feature Importance by Effect Size", fontsize=16)
plt.xlabel("Effect Size (Cohen's d)", fontsize=14)
plt.ylabel("Feature", fontsize=14)
plt.legend(title="Statistically Significant")
plt.tight_layout()
plt.show()

### 4.3 Correlation Matrix


In [ ]:
corr_matrix = df_clean[numerical_cols].corr()

In [ ]:
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(
    corr_matrix,
    mask=mask,
    annot=True,
    cmap=cmap,
    linewidths=0.5,
    fmt=".2f",
    vmin=-1,
    vmax=1,
    center=0,
    square=True,
    cbar_kws={"shrink": 0.8},
)

plt.title("Correlation Matrix of Numerical Features", fontsize=16)
plt.tight_layout()
plt.show()

### 4.4 Check for High Correlations


In [ ]:
high_corr_threshold = 0.7
high_corr_pairs = []

for i in range(len(corr_matrix.columns)):
    for j in range(i + 1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) >= high_corr_threshold:
            high_corr_pairs.append(
                {
                    "Feature 1": corr_matrix.columns[i],
                    "Feature 2": corr_matrix.columns[j],
                    "Correlation": corr_matrix.iloc[i, j],
                }
            )

In [ ]:
if high_corr_pairs:
    print("\nHigh Correlation Pairs (potential multicollinearity):")
    display(pd.DataFrame(high_corr_pairs))

else:
    print("\nNo high correlation pairs found.")

## 5.0 Feature Engineering


In [ ]:
df_featured = df_clean.copy()

### 5.1 Creating the Features


Feature 1: Customer Lifetime Value (CLV)


In [ ]:
df_featured["CLV"] = df_featured["tenure"] * df_featured["MonthlyCharges"]

Feature 2: Service Count


In [ ]:
service_cols = [
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
]

In [ ]:
df_featured["ServiceCount"] = 0
for col in service_cols:
    df_featured["ServiceCount"] += (
        (df_featured[col] != "No")
        & (df_featured[col] != "No internet service")
        & (df_featured[col] != "No phone service")
    ).astype(int)

Feature 3: Average Spend Per Service


In [ ]:
df_featured["AvgSpendPerService"] = df_featured.apply(
    lambda x: x["MonthlyCharges"] / max(x["ServiceCount"], 1), axis=1
)

Feature 4: Tenure Groups


In [ ]:
bins = [0, 12, 24, 36, 48, 60, np.inf]
labels = [
    "0-12 months",
    "13-24 months",
    "25-36 months",
    "37-48 months",
    "49-60 months",
    "60+ months",
]

In [ ]:
df_featured["TenureGroup"] = pd.cut(df_featured["tenure"], bins=bins, labels=labels)

Feature 5: Has Security Services (both OnlineSecurity and TechSupport)


In [ ]:
df_featured["HasSecurityServices"] = (
    (df_featured["OnlineSecurity"] == "Yes") & (df_featured["TechSupport"] == "Yes")
).astype(int)

Feature 6: Contract Duration in Months


In [ ]:
contract_map = {"Month-to-month": 1, "One year": 12, "Two year": 24}
df_featured["ContractDuration"] = df_featured["Contract"].map(contract_map)

Feature 7: Tenure to Contract Ratio


In [ ]:
df_featured["TenureContractRatio"] = (
    df_featured["tenure"] / df_featured["ContractDuration"]
)

In [ ]:
display(
    df_featured[
        [
            "CLV",
            "ServiceCount",
            "AvgSpendPerService",
            "TenureGroup",
            "HasSecurityServices",
            "ContractDuration",
            "TenureContractRatio",
        ]
    ]
)

### 5.2 Visualization of Engineered Features


In [ ]:
def analyze_engineered_feature(df, feature, is_categorical=False):
    """
    Analyze an engineered feature's relationship with churn

    Args:
        df: DataFrame with the data
        feature: Feature name to analyze
        is_categorical: Whether the feature is categorical
    """
    if is_categorical:
        churn_rate = (
            df.groupby(feature)[CONFIG["target_column"]]
            .apply(lambda x: (x == CONFIG["positive_class"]).mean() * 100)
            .reset_index()
        )
        churn_rate.columns = [feature, "Churn Rate (%)"]

        fig = px.bar(
            churn_rate,
            x=feature,
            y="Churn Rate (%)",
            title=f"Churn Rate by {feature}",
            color="Churn Rate (%)",
            color_continuous_scale="RdBu_r",
            template="plotly_white",
        )

        fig.update_layout(
            xaxis_title=feature,
            yaxis_title="Churn Rate (%)",
            title_font_size=16,
            title_x=0.5,
        )
    else:
        df_temp = df.copy()
        bins = 5
        bin_labels = [f"Bin {i+1}" for i in range(bins)]
        df_temp[f"{feature}_bin"], bin_edges = pd.cut(
            df_temp[feature], bins=bins, labels=bin_labels, retbins=True
        )

        # Calculate churn rate by bin
        churn_rate = (
            df_temp.groupby(f"{feature}_bin")[CONFIG["target_column"]]
            .apply(lambda x: (x == CONFIG["positive_class"]).mean() * 100)
            .reset_index()
        )
        churn_rate.columns = [f"{feature}_bin", "Churn Rate (%)"]

        # Add range information to bin labels for better interpretation
        bin_ranges = [
            f"{bin_edges[i]:.1f}-{bin_edges[i+1]:.1f}"
            for i in range(len(bin_edges) - 1)
        ]

        bin_mapping = dict(zip(bin_labels, bin_ranges))
        churn_rate["Bin Range"] = churn_rate[f"{feature}_bin"].map(bin_mapping)

        fig = px.bar(
            churn_rate,
            x=f"{feature}_bin",
            y="Churn Rate (%)",
            title=f"Churn Rate by {feature} (Quintiles)",
            color="Churn Rate (%)",
            color_continuous_scale="RdBu_r",
            template="plotly_white",
            hover_data=["Bin Range"],
        )

        fig.update_layout(
            xaxis_title=feature,
            yaxis_title="Churn Rate (%)",
            title_font_size=16,
            title_x=0.5,
        )

    fig.show()

    if not is_categorical:
        churned = df[df[CONFIG["target_column"]] == CONFIG["positive_class"]][feature]
        retained = df[df[CONFIG["target_column"]] != CONFIG["positive_class"]][feature]

        # Mann-Whitney U test
        u_stat, p_value = stats.mannwhitneyu(churned, retained)

        # Effect size
        mean_diff = churned.mean() - retained.mean()
        pooled_std = np.sqrt((churned.std() ** 2 + retained.std() ** 2) / 2)
        cohens_d = abs(mean_diff / pooled_std)

        if cohens_d < 0.2:
            effect = "negligible"

        elif cohens_d < 0.5:
            effect = "small"

        elif cohens_d < 0.8:
            effect = "medium"

        else:
            effect = "large"

        print(f"Statistical Analysis for {feature}:")
        print(f"- Mann-Whitney U Test: U = {u_stat:.2f}, p-value = {p_value:.4f}")
        print(f"- Effect Size (Cohen's d): {cohens_d:.4f} - {effect} effect")
        print(f"- Mean for Churned Customers: {churned.mean():.2f}")
        print(f"- Mean for Retained Customers: {retained.mean():.2f}")

In [ ]:
numerical_engineered = [
    "CLV",
    "ServiceCount",
    "AvgSpendPerService",
    "TenureContractRatio",
]

for feature in numerical_engineered:
    analyze_engineered_feature(df_featured, feature, is_categorical=False)

In [ ]:
analyze_engineered_feature(df_featured, "TenureGroup", is_categorical=True)

In [ ]:
analyze_engineered_feature(df_featured, "HasSecurityServices", is_categorical=True)

## 6.0 Customer Segmentation Analysis


In [ ]:
cluster_features = [
    "tenure",
    "MonthlyCharges",
    "CLV",
    "ServiceCount",
    "AvgSpendPerService",
]

In [ ]:
X_cluster = df_featured[cluster_features].copy()

Standardize the data


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cluster)

Determine the optimal number of clusters using the elbow method


In [ ]:
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=CONFIG["random_seed"], n_init=10)
    kmeans.fit(X_scaled)
    wcss.append(kmeans.inertia_)

Plot the elbow curve


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), wcss, marker="o", linestyle="-")

plt.title("Elbow Method for Optimal k", fontsize=16)
plt.xlabel("Number of Clusters", fontsize=14)
plt.ylabel("WCSS (Within-Cluster Sum of Squares)", fontsize=14)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

Apply K-means with optimal number of clusters


In [ ]:
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=CONFIG["random_seed"], n_init=10)

In [ ]:
clusters = kmeans.fit_predict(X_scaled)
df_featured["Cluster"] = clusters

Add cluster labels to the dataset


In [ ]:
cluster_analysis = (
    df_featured.groupby("Cluster")
    .agg(
        {
            "tenure": "mean",
            "MonthlyCharges": "mean",
            "TotalCharges": "mean",
            "CLV": "mean",
            "ServiceCount": "mean",
            "AvgSpendPerService": "mean",
            "HasSecurityServices": "mean",
            "ContractDuration": "mean",
            "TenureContractRatio": "mean",
            CONFIG["target_column"]: lambda x: (x == CONFIG["positive_class"]).mean()
            * 100,
        }
    )
    .round(2)
)

Rename the Churn column for clarity

In [ ]:
cluster_analysis.rename(
    columns={CONFIG["target_column"]: "Churn Rate (%)"}, inplace=True
)

Get cluster sizes

In [ ]:
cluster_sizes = df_featured["Cluster"].value_counts().reset_index()
cluster_sizes.columns = ["Cluster", "Count"]
cluster_sizes["Percentage"] = (cluster_sizes["Count"] / len(df_featured) * 100).round(2)

### 6.1 Display Cluster Analysis

In [ ]:
display(cluster_analysis)

In [ ]:
display(cluster_sizes)

### 6.2 Visualize Clusters using PCA

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

In [ ]:
pca_df = pd.DataFrame({
    'PCA1': X_pca[:, 0],
    'PCA2': X_pca[:, 1],
    'Cluster': clusters.astype(str),  
    'Churn': df_featured[CONFIG['target_column']]
})

In [ ]:
fig = px.scatter(
    pca_df,
    x="PCA1",
    y="PCA2",
    color="Cluster",
    symbol="Churn",
    title="Customer Segments Visualization",
    labels={
        "PCA1": f"Principal Component 1 ({pca.explained_variance_ratio_[0]:.2%})",
        "PCA2": f"Principal Component 2 ({pca.explained_variance_ratio_[1]:.2%})",
    },
    template="plotly_white",
)

fig.update_layout(
    title_font_size=16, title_x=0.5, legend_title_text="Cluster", width=900, height=700
)

fig.show()

### 6.3 Analyze Churn Rate by Cluster

In [ ]:
churn_by_cluster = (
    df_featured.groupby("Cluster")[CONFIG["target_column"]]
    .apply(lambda x: (x == CONFIG["positive_class"]).mean() * 100)
    .reset_index()
)

churn_by_cluster.columns = ["Cluster", "Churn Rate (%)"]

In [ ]:
churn_by_cluster["Cluster"] = churn_by_cluster["Cluster"].astype(str)

In [ ]:
fig = px.bar(
    churn_by_cluster,
    x="Cluster",
    y="Churn Rate (%)",
    title="Churn Rate by Customer Segment",
    color="Churn Rate (%)",
    color_continuous_scale="RdBu_r",
    template="plotly_white",
)

fig.update_layout(
    title_font_size=16,
    title_x=0.5,
    xaxis_title="Customer Segment",
    yaxis_title="Churn Rate (%)",
)

fig.update_traces(texttemplate="%{y:.1f}%", textposition="outside")
fig.show()